In [241]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import re

In [242]:
course_df_original = pd.read_csv('../data/courses.csv').fillna('')
course_df = course_df_original.drop(['course_id', 'teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df['description'] = course_df['description'].replace('([\<]).*?([\>])','',regex=True)
course_df = course_df.replace('\n', '',regex=True)
course_df = course_df.replace('[\d_]', '',regex=True).astype(str)
course_df.insert(0, 'course_id', course_df_original['course_id'])
course_df[:2]

,course_id,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","生活品味更多生活品味,生活品味護膚保養與化妝","更多生活品味,護膚保養與化妝",少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好...,不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️ 雖然課程當中會帶到相關彩妝技巧，不過內容偏...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計平面設計,藝術繪畫與插畫","Illustrator/以拉,配色技巧",這是一個有趣的小技巧，主要利用illustrator裡面的&quot;分割“這個功能，我已經...,可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


In [243]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix

In [244]:
info = []
for i in range(course_df.shape[0]):
    t = course_df.loc[i, :].values.flatten().tolist()
    info.append([t[0], ' '.join(t)])
info = pd.DataFrame(info, columns = ['course_id', 'text'])
info[:5]

,course_id,text
0,61888e868f154b000781b45a,61888e868f154b000781b45a 少女人妻華麗變身：七大妝容七彩的夢幻樂園 ...
1,54d5a117065a7e0e00725ac0,54d5a117065a7e0e00725ac0 幾何圖形分割 X 色塊組合 從學生時代開始...
2,54d5d9952246e60a009ec571,54d5d9952246e60a009ec571 數位拼貼的手感 自由工作者，致力於品牌視覺...
3,54d7148a2246e60a009ec588,54d7148a2246e60a009ec588 Line 的貼圖自己動手做！ 我是Dann...
4,5513e92b38239d10005778e1,5513e92b38239d10005778e1 為申請學校或工作寫好英文自傳 在北美長大，...


In [245]:
ws = WS('./ckip/data')

C:\Users\Vincent\AppData\Local\Programs\Python\Python310\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [246]:
tfidf_ml = TfidfVectorizer()
tfidf_ml.fit(info.text)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names_out(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()

C:\Users\Vincent\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [247]:
len(dictionary.keys())

174321

In [248]:
tfidf_feature[:10]

['54d5a117065a7e0e00725ac0',
 '54d5d9952246e60a009ec571',
 '54d7148a2246e60a009ec588',
 '54f1268f4ec3c809002e4a29',
 '551171a938239d1000577864',
 '551198a738239d1000577870',
 '5513e92b38239d10005778e1',
 '551a6be023774e0a001eb20c',
 '55307d94d530a90a00a3a896',
 '55599687dfe21b0a00e776d9']

In [249]:
# #Construct the required TF-IDF matrix by fitting and transforming the data
# vectorizer = TfidfVectorizer(stop_words= 'english')
# tfidf_matrix = vectorizer.fit_transform(info)

# #print(tfidf.vocabulary_)
# tfidf_matrix_nd = tfidf_matrix.T.toarray()
# #np.save('tfidf_embedding/item_embeddings_english.npy', tfidf_matrix_nd)
# df = pd.DataFrame(tfidf_matrix_nd, index=vectorizer.get_feature_names_out ())

In [250]:
from gensim.models.word2vec import Word2Vec

w2v_model = Word2Vec(info.text.apply(lambda text: text.split()))
w2v_vocab = list(w2v_model.wv.index_to_key)
print(w2v_model)

Word2Vec<vocab=1314, vector_size=100, alpha=0.025>


In [251]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in info.text.apply(lambda text: text.split()):
    text_vect = np.zeros(100)
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)



建立模型時間:  0:03:34.946432


In [252]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    text = re.sub(' +', '',text)
    text = ws([text], sentence_segmentation=True)
    text = [x for l in text for x in l]
    return text

In [253]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect, top_k = 50):
    text = Preprocess(text)
    text_vect = np.zeros(100) # w2v size
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]
    tmp_top_k = info[['text']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]]
    tmp_top_k['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_k



In [258]:
starttime = datetime.datetime.now()

newtext = '職場技能'
result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)
result

搜尋推薦時間:  0:00:00.016901


,text,similarity_score
249,5b2088a6567cc1001e40f891 韓式豆沙裱花-特殊造型蛋糕 我是萬萬老師，...,0.0
247,5b61928a8011d1001e356102 批判到創新：不是研究生也能學的研究方法 聯...,0.0
246,5b9f4fcb88e4f8001ec68c3d 標標標標標準字 - 從生活創意到邏輯實踐 ...,0.0
245,5bc0dc7915aece001e61203f Java 與 Kotlin 一起學 : 程...,0.0
244,5b8a4d1d3aeadb001e5cf364 藍曬技法 - 初學者的氰版顯影 Hel...,0.0
243,5a1fd1b7e711cc001ede7a2d Python 資料分析＆算法交易 Must...,0.0
242,5b3cdd6ed03140001eebeadc 零基礎邁向區塊鏈工程師：Solidity ...,0.0
241,5b8e5fb4de22ba001ee7a572 職場英文大補帖｜上班族的 Email 寫作...,0.0
240,5af1c528b44939001e4637b6 玩轉 Unity! 零基礎打造自己的 D ...,0.0
239,5b3b080baa5ff8001e22316e Evernote，你的無壓工作術 你的數位...,0.0


In [255]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
test_unseen

SyntaxError: invalid syntax (3193057967.py, line 1)